In [1]:
### Image preprocessing for TRAINING and TESTING DATA

In [2]:
# Specifing the folder where the image are present
traning_image='../FaceDetection/Face Images/Final Training Images'


In [3]:
len(traning_image)

50

In [4]:
from keras.preprocessing.image import ImageDataGenerator

train_data_generator=ImageDataGenerator(
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True

)
# Defining pre-processing transformations on raw images of training data
# These hyper parameters helps to generate slightly twisted versions
# of the original image, which leads to a better model, since it learns
# on the good and bad mix of images

test_data_generator=ImageDataGenerator()
# Defining pre-processing transformations on raw images of testing data
# No transformations are done on the testing images



#Generating the traning data
traning_set=train_data_generator.flow_from_directory(
    traning_image,
    target_size=(64,64),
    batch_size=32,
    class_mode='categorical'
)

#Generating the test data
test_set=train_data_generator.flow_from_directory(
    traning_image,
    target_size=(64,64),
    batch_size=32,
    class_mode='categorical'
)

#Printing the class label for each face
test_set.class_indices



Found 244 images belonging to 16 classes.
Found 244 images belonging to 16 classes.


{'face1': 0,
 'face10': 1,
 'face11': 2,
 'face12': 3,
 'face13': 4,
 'face14': 5,
 'face15': 6,
 'face16': 7,
 'face2': 8,
 'face3': 9,
 'face4': 10,
 'face5': 11,
 'face6': 12,
 'face7': 13,
 'face8': 14,
 'face9': 15}

In [5]:

training_class_indices=traning_set.class_indices
training_class_indices


{'face1': 0,
 'face10': 1,
 'face11': 2,
 'face12': 3,
 'face13': 4,
 'face14': 5,
 'face15': 6,
 'face16': 7,
 'face2': 8,
 'face3': 9,
 'face4': 10,
 'face5': 11,
 'face6': 12,
 'face7': 13,
 'face8': 14,
 'face9': 15}

In [6]:
#creating a mapping for index and face names

result_indices_map={}


# Here first line of for loop is getting the indices values and key i.e:) {'face1':0}
for face_value,face_name in zip(training_class_indices.values(),training_class_indices.keys()):
    result_indices_map[face_value]=face_name  # Here to mapping the values to keys and keys to  values i.e: {0:'face1'}
    
    
# saving the map for feature reference
import pickle

with open('result_indices_map.pkl','wb') as fileWriteStream:
    pickle.dump(result_indices_map,fileWriteStream)

# Here we get recent mapping and corresponding face name of it
print('Mapping of Face and its ID:',result_indices_map)


# Number of neurons for the output layer is equal to the number of faces
output_neurons=len(result_indices_map)
print('Number of neurons:',output_neurons)


Mapping of Face and its ID: {0: 'face1', 1: 'face10', 2: 'face11', 3: 'face12', 4: 'face13', 5: 'face14', 6: 'face15', 7: 'face16', 8: 'face2', 9: 'face3', 10: 'face4', 11: 'face5', 12: 'face6', 13: 'face7', 14: 'face8', 15: 'face9'}
Number of neurons: 16


In [7]:
# Now Lets start to build CNN model

# Before creating the CNN we have to import some of the importent lib

In [8]:
from keras.models import Sequential
from keras.layers import Convolution2D,MaxPool2D,Flatten,Dense

## Initializing the CNN model
model=Sequential()

''' STEP--1 Convolution
# Adding the first layer of CNN
# we are using the format (64,64,3) because we are using TensorFlow backend
# It means 3 matrix of size (64X64) pixels representing Red, Green and Blue components of pixels'''
model.add(Convolution2D(
    32,
    kernel_size=(5,5),
    strides=(1,1),
    input_shape=(64,64,3),
    activation='relu'
))

# Max Pooling 
model.add(MaxPool2D(pool_size=(2,2)))


# ADDITIONAL LAYER of CONVOLUTION for better accuracy

model.add(Convolution2D(
    64,
    kernel_size=(5,5),
    strides=(1,1),
    activation='relu'

))
model.add(MaxPool2D(pool_size=(2,2)))

# Flattern Layer

model.add(Flatten())

# Fully Connected NN
model.add(Dense(64,activation='relu'))

model.add(Dense(output_neurons,activation='softmax'))

model.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

import time
# Measuring the time taken by the model to train
Start_time=time.time()

model.fit(
    traning_set,
    epochs=10,
    validation_data=test_set,
)

End_time=time.time()

print('Total time takes:{}',round((End_time-Start_time)/60),'Minutes.')

Epoch 1/10
8/8 [==============================] - 4s 369ms/step - loss: 112.5598 - accuracy: 0.0574 - val_loss: 2.7473 - val_accuracy: 0.1598
Epoch 2/10
8/8 [==============================] - 3s 331ms/step - loss: 2.6651 - accuracy: 0.1393 - val_loss: 2.6327 - val_accuracy: 0.2295
Epoch 3/10
8/8 [==============================] - 3s 340ms/step - loss: 2.5318 - accuracy: 0.2664 - val_loss: 1.9997 - val_accuracy: 0.4303
Epoch 4/10
8/8 [==============================] - 3s 352ms/step - loss: 1.8174 - accuracy: 0.4549 - val_loss: 1.2456 - val_accuracy: 0.6230
Epoch 5/10
8/8 [==============================] - 3s 347ms/step - loss: 1.1499 - accuracy: 0.6762 - val_loss: 0.6414 - val_accuracy: 0.8279
Epoch 6/10
8/8 [==============================] - 3s 333ms/step - loss: 0.6829 - accuracy: 0.7869 - val_loss: 0.6841 - val_accuracy: 0.7951
Epoch 7/10
8/8 [==============================] - 3s 342ms/step - loss: 0.5342 - accuracy: 0.8402 - val_loss: 0.3441 - val_accuracy: 0.9262
Epoch 8/10
8/8 [==

In [18]:
# Now lets test our CNN model with unseen images

import numpy as np
from keras.preprocessing import image

Image_test_path='../FaceDetection/Face Images/Final Testing Images/face4/3face4.jpg'

test_image=image.load_img(Image_test_path,target_size=(64,64))
test_image=image.img_to_array(test_image)

In [19]:
# axis=0 specifies that you want to add a new dimension at the beginning of the array. 
#This effectively adds a new "batch" dimension. 
test_image=np.expand_dims(test_image,axis=0)

result=model.predict(test_image,verbose=0)

print('Prediction Face:',result_indices_map[np.argmax(result)])

#yeah.the model has predicted this face correctly.Lets try another image 

Prediction Face: face4


In [20]:
Image_test_path_sec='../FaceDetection/Face Images/Final Testing Images/face11/1face11.jpg'

test_image_sec=image.load_img(Image_test_path_sec,target_size=(64,64))
test_image_sec=image.img_to_array(test_image_sec)

test_image_sec=np.expand_dims(test_image_sec,axis=0) 
# axis=0 specifies that you want to add a new dimension at the beginning of the array. 
#This effectively adds a new "batch" dimension.

result=model.predict(test_image_sec,verbose=0)

print('2 nd Prediction Face:',result_indices_map[np.argmax(result)])


2 nd Prediction Face: face11
